<h1>Python algorithms part 3<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Задача-«Обойди-дерево»" data-toc-modified-id="Задача-«Обойди-дерево»-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Задача «Обойди дерево»</a></span></li><li><span><a href="#Камни-с-Марса" data-toc-modified-id="Камни-с-Марса-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Камни с Марса</a></span></li><li><span><a href="#Динамическое-программирование:-посылка-на-Марс" data-toc-modified-id="Динамическое-программирование:-посылка-на-Марс-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Динамическое программирование: посылка на Марс</a></span></li></ul></div>

Ваша задача — дописать функцию, которая принимает на вход бинарное дерево (генеалогическое древо) и возвращает сумму значений space_experience всех узлов этого дерева.

### Задача «Обойди дерево»

In [1]:
from __future__ import annotations
from typing import Optional


class Spaceman:

    def __init__(
            self,
            name: str,
            space_experience: int,
            father: Optional[Spaceman] = None,
            mother: Optional[Spaceman] = None,
    ):
        self.name = name
        self.space_experience = space_experience
        self.father = father
        self.mother = mother


def count_dynasty_experience(spaceman: Spaceman) -> int:
    total_experience = spaceman.space_experience
    # Доработайте функцию, чтобы она считала
    # суммарный опыт династии космонавтов.
    if spaceman.father:
        total_experience += count_dynasty_experience(spaceman.father)
    if spaceman.mother:
        total_experience += count_dynasty_experience(spaceman.mother)
    return total_experience


yu_a_tatarin = Spaceman(
    name='Юрий Алексеевич Татарин',
    space_experience=10,
    father=Spaceman(
        name='Алексей Михайлович Татарин',
        space_experience=25,
        mother=Spaceman(
            name='Евгения Владимировна Беляева',
            space_experience=1
        )
    ),
    mother=Spaceman('Ангелина Васильевна Черенкова', 5)
)
result = count_dynasty_experience(yu_a_tatarin)
print(result)

41


### Камни с Марса

При отправке марсианской миссии множество научных лабораторий и музеев сделали заказы на доставку образцов марсианских горных пород.

В каждом заказе обозначен минимальный вес требуемого образца: заказчик готов получить образец указанного веса или тяжелее, но никак не меньше.

Марсианские булыжники были доставлены на Землю, но ни их количество, ни вес не соответствуют заказам в точности:

8 5 5 8 6 9 8 2 4 7   # Заказано 10 образцов;

Каждый элемент - вес заказанного образца.

9 8 1 1 1 5 10 8      # Доставлено 8 образцов;

Каждый элемент - вес доставленного образца.
                      
Ваша задача: написать программу, которая распределит образцы между заказчиками оптимальным образом — так, чтобы выполнить требования максимального числа заказчиков.

Требование заказчика считается выполненным, если он получит образец, вес которого равен заказанному или превышает заказанный вес.

Программа должна вернуть число заказчиков, которые получили образцы, соответствующие их требованиям.

8 5 5 8 6 9 8 2 4 7   # Заказано.

9 8 1 1 1 5 10 8      # Доставлено.

8 5 5 8  6 9 8 2 4 7  # Доставленные образцы, распределённые между заказчиками

| | | |  |            # (один из возможных вариантов).

9 8 5 10 8            # Три образца весом 1 кг остались невостребованными.

Результат (число заказчиков, получивших желаемое): 5

Булыжники нельзя дробить на части, а каждый заказчик может получить только по одному образцу.


In [4]:
def orders_distribution(orders_arr, delivery_arr):
    counter = 0
    orders_arr = sorted(orders_arr)
    delivery_arr = sorted(delivery_arr)
    i, j = 0, 0

    while i < len(orders_arr) and j < len(delivery_arr):
        if orders_arr[i] <= delivery_arr[j]:
            counter += 1
            i += 1
        j += 1
    return counter

orders_qty  = int(input().strip())
orders_weight = [int(number) for number in input().split()]
delivery_qty = int(input().strip())
delivery_weight = [int(number) for number in input().split()]


print(orders_distribution(orders_weight, delivery_weight))

10
8 5 5 8 6 9 8 2 4 7
8
9 8 1 1 1 5 10 8
5


### Динамическое программирование: посылка на Марс

In [5]:
from pprint import pprint


def backpack_problem_solution(
        tools: list[tuple[str, int, int]], capacity: int
) -> str:
    # Сохраняем количество приборов в переменную.
    items_count = len(tools)
    # Создаём таблицу. В каждой ячейке должны храниться число и список:
    # количество экспериментов и список приборов. Для простоты подсчётов 
    # добавим строку с отсутствием рассматриваемых приборов
    # и столбец с нулевой вместимостью контейнера.
    table = [
        [[0, []] for _ in range(capacity + 1)] for _ in range(items_count + 1)
    ]
    # Для каждого прибора:
    for row_number in range(1, items_count + 1):
        # Распаковываем кортеж с данными о приборе.
        name, mass, experiments = tools[row_number - 1]
        # Для контейнера вместимостью от 1 и до максимальной вместимости:
        for volume in range(1, capacity + 1):
            # Если вес прибора меньше или равен 
            # вместимости рассматриваемого контейнера.
            if mass <= volume:
                # Считаем количество экспериментов для текущего прибора
                # плюс наилучшее решение для оставшейся вместимости 
                # из предыдущей строки.
                total_experiments_with_current_tool = (
                    experiments + table[row_number - 1][volume - mass][0]
                )
                # Количество экспериментов 
                # в текущей колонке на предыдущей строке:
                previous_result = table[row_number - 1][volume][0]
                # Если результат с текущим прибором лучше:
                if total_experiments_with_current_tool > previous_result:
                    # Записываем количество экспериментов.
                    table[row_number][volume][0] = (
                        total_experiments_with_current_tool
                    )
                    # Копируем список приборов из предыдущей строки 
                    # из ячейки, равной оставшейся вместимости.
                    table[row_number][volume][1] = list.copy(
                        table[row_number - 1][volume - mass][1]
                    )
                    # Добавляем к списку приборов
                    # имя текущего прибора.
                    table[row_number][volume][1].append(name)
                else:
                    # Если результат с рассматриваемым прибором 
                    # хуже или такой же -
                    # переносим результат с предыдущей строки.
                    table[row_number][volume] = table[row_number - 1][volume]
            else:
                # Если масса рассматриваемого прибора 
                # больше вместимости ячейки - 
                # переносим результат с предыдущей строки.
                table[row_number][volume] = table[row_number - 1][volume]
    # Распечатываем таблицу для проверки.
    pprint(table)
    # Возвращаем строку с названиями приборов
    # из нижней правой ячейки таблицы, через запятую.
    return ', '.join(table[-1][-1][-1])    


if __name__ == '__main__':
    # [('название прибора', масса, количество экспериментов)]
    tools = [
        ('гигрометр', 1, 3),
        ('масс-спектрометр', 4, 6),
        ('нивелир', 3, 4),
        ('интерферометр', 1, 4)
    ]
    result = backpack_problem_solution(tools, 4)
    print(result)
 


[[[0, []], [0, []], [0, []], [0, []], [0, []]],
 [[0, []],
  [3, ['гигрометр']],
  [3, ['гигрометр']],
  [3, ['гигрометр']],
  [3, ['гигрометр']]],
 [[0, []],
  [3, ['гигрометр']],
  [3, ['гигрометр']],
  [3, ['гигрометр']],
  [6, ['масс-спектрометр']]],
 [[0, []],
  [3, ['гигрометр']],
  [3, ['гигрометр']],
  [4, ['нивелир']],
  [7, ['гигрометр', 'нивелир']]],
 [[0, []],
  [4, ['интерферометр']],
  [7, ['гигрометр', 'интерферометр']],
  [7, ['гигрометр', 'интерферометр']],
  [8, ['нивелир', 'интерферометр']]]]
нивелир, интерферометр
